## Preprocessing


In [1]:
import os
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Ensure you have the NLTK stopwords downloaded
import nltk
nltk.download('stopwords')

# def preprocess_text(text):
#     # Remove punctuation and extra spaces
#     text = re.sub(r'[^\w\s]', '', text)
#     text = re.sub(r'\s+', ' ', text).strip()
#     return text

# def preprocess_file(input_file, output_file):
#     with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
#         for line in infile:
#             preprocessed_line = preprocess_text(line)
#             outfile.write(preprocessed_line + '\n')

# def preprocess_dataset(input_directory, output_directory):
#     if not os.path.exists(output_directory):
#         os.makedirs(output_directory)

#     for filename in os.listdir(input_directory):
#         if filename.endswith('.en'):
#             input_file = os.path.join(input_directory, filename)
#             output_file = os.path.join(output_directory, filename)

#             preprocess_file(input_file, output_file)

# if __name__ == '__main__':
#     input_directory = "data"
#     output_directory = "your_output_directory"

#     preprocess_dataset(input_directory, output_directory)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Data-Handling

In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import random

from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

#read input files
trainEn = "/content/drive/My Drive/train.en"
trainFr = "/content/drive/My Drive/train.fr"
testEn = "/content/drive/My Drive/test.en"
testFr = "/content/drive/My Drive/test.fr"
devEn = "/content/drive/My Drive/dev.en"
devFr = "/content/drive/My Drive/dev.fr"

# Initialize a list to store the sentences
trainDataEN = []
trainDataFR = []
testDataEN = []
testDataFR = []
devDataEN = []
devDataFR = []

# Read the file line by line
with open(trainEn, "r") as file:
    for line in file:
        # Remove leading and trailing whitespace and append the line as a sentence
        sentence = line.strip()
        trainDataEN.append(sentence)

with open(trainFr, "r") as file:
    for line in file:
        sentence = line.strip()
        trainDataFR.append(sentence)

with open(testEn, "r") as file:
    for line in file:
        sentence = line.strip()
        testDataEN.append(sentence)

with open(testFr, "r") as file:
    for line in file:
        sentence = line.strip()
        testDataFR.append(sentence)

with open(devEn, "r") as file:
    for line in file:
        sentence = line.strip()
        devDataEN.append(sentence)

with open(devFr, "r") as file:
    for line in file:
        sentence = line.strip()
        devDataFR.append(sentence)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Dataloader

In [69]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from torchtext.data import get_tokenizer
from torchtext.vocab import Vocab
import spacy
from spacy.lang.fr.examples import sentences
import collections



In [ ]:
!python -m spacy download en_core_web_sm


In [ ]:
!python -m spacy download fr_core_news_sm


In [70]:
# Initialize vocabularies with special tokens
special_tokens = ['<pad>', '<sos>', '<eos>', '<unk>']
vocab_en = {token: index for index, token in enumerate(special_tokens)}
vocab_fr = {token: index for index, token in enumerate(special_tokens)}

# Function to build vocabulary
def build_vocab(data, vocab):
    for sentence in data:
        words = sentence.split()  # Split the sentence into words
        for word in words:
            if word not in vocab:
                vocab[word] = len(vocab)

# Build vocabularies from training data
build_vocab(trainDataEN, vocab_en)
build_vocab(trainDataFR, vocab_fr)



In [85]:
print("English Vocabulary Size:", len(vocab_en))
print("French Vocabulary Size:", len(vocab_fr))
src_vocab_size = len(vocab_en)
trg_vocab_size = len(vocab_fr)


English Vocabulary Size: 27004
French Vocabulary Size: 35348


In [77]:
# Function to convert sentences to numerical indices
def sentences_to_indices(sentences, vocab):
    indices = []
    for sentence in sentences:
        words = sentence.split()
        sentence_indices = [vocab.get(word, vocab['<unk>']) for word in words]
        indices.append(sentence_indices)
    return indices

# Convert data to numerical indices
trainDataEN_indices = sentences_to_indices(trainDataEN, vocab_en)
trainDataFR_indices = sentences_to_indices(trainDataFR, vocab_fr)
testDataEN_indices = sentences_to_indices(testDataEN, vocab_en)
testDataFR_indices = sentences_to_indices(testDataFR, vocab_fr)
devDataEN_indices = sentences_to_indices(devDataEN, vocab_en)
devDataFR_indices = sentences_to_indices(devDataFR, vocab_fr)


In [93]:
# Define your custom dataset for training
class CustomDataset(Dataset):
    def __init__(self, source_indices, target_indices, max_seq_length):
        self.source_indices = source_indices
        self.target_indices = target_indices
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.source_indices)

    def __getitem__(self, idx):
        source_sequence = self.source_indices[idx]
        target_sequence = self.target_indices[idx]

        # Pad sequences to have the same length
        source_sequence = source_sequence[:self.max_seq_length]
        target_sequence = target_sequence[:self.max_seq_length]

        # Pad with <pad> token if the sequence is shorter
        while len(source_sequence) < self.max_seq_length:
            source_sequence.append(vocab_en['<pad>'])
        while len(target_sequence) < self.max_seq_length:
            target_sequence.append(vocab_fr['<pad>'])

        return source_sequence, target_sequence

# Create instances of your custom dataset
max_seq_length = 25
train_dataset = CustomDataset(trainDataEN_indices, trainDataFR_indices, max_seq_length)
dev_dataset = CustomDataset(devDataEN_indices, devDataFR_indices, max_seq_length)
test_dataset = CustomDataset(testDataEN_indices, testDataFR_indices, max_seq_length)


In [94]:
# Define data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [95]:
print(train_dataset[3])

([41, 42, 28, 21, 43, 7, 44, 21, 31, 45, 46, 47, 48, 49, 50, 28, 51, 52, 53, 47, 37, 21, 29, 54, 55], [41, 42, 43, 44, 45, 27, 46, 47, 48, 18, 49, 50, 29, 51, 18, 52, 53, 37, 54, 19, 30, 55, 0, 0, 0])


## Self Attention

In [19]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(embed_size, embed_size)
        self.keys = nn.Linear(embed_size, embed_size)
        self.queries = nn.Linear(embed_size, embed_size)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask):
        # Get number of training examples
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        values = self.values(values)  # (N, value_len, embed_size)
        keys = self.keys(keys)  # (N, key_len, embed_size)
        queries = self.queries(query)  # (N, query_len, embed_size)

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = queries.reshape(N, query_len, self.heads, self.head_dim)

        # Einsum does matrix mult. for query*keys for each training example
        # with every other training example, don't be confused by einsum

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (N, query_len, heads, heads_dim),
        # keys shape: (N, key_len, heads, heads_dim)
        # energy: (N, heads, query_len, key_len)

        # Mask padded indices so their weights become 0
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        # attention shape: (N, heads, query_len, key_len)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out after matrix multiply: (N, query_len, heads, head_dim), then
        # we reshape and flatten the last two dimensions.

        out = self.fc_out(out)
        # Linear layer doesn't modify the shape, final shape will be
        # (N, query_len, embed_size)

        return out


## Transformer

In [20]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out


### Encoder

In [21]:
class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))
        )

        # In the Encoder the query, key, value are all the same, it's in the
        # decoder this will change. This might look a bit odd in this case.
        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out


### Decoder

In [22]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out


In [23]:
class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)

        return out


### back to trans

In [24]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=512,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        device="cpu",
        max_length=100,
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out


## Training

In [113]:
import math
from tqdm import tqdm


In [96]:
# Initialize your Transformer model
src_pad_idx = vocab_en['<pad>']
trg_pad_idx = vocab_fr['<pad>']

model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx)

# Define your optimizer and loss function
lr=0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

# Move the model to the GPU, if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Transformer(
  (encoder): Encoder(
    (word_embedding): Embedding(27004, 512)
    (position_embedding): Embedding(100, 512)
    (layers): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): SelfAttention(
          (values): Linear(in_features=512, out_features=512, bias=True)
          (keys): Linear(in_features=512, out_features=512, bias=True)
          (queries): Linear(in_features=512, out_features=512, bias=True)
          (fc_out): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (feed_forward): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): ReLU()
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (dropout): Dropout(p=0, inplace=False)
      )
    )
    (dropout): Dropout(p=0, inplace=False)
  )
  (decoder): Decoder(

In [116]:
# Define the number of epochs and other hyperparameters
num_epochs = 4
learning_rate = 0.001

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    # Wrap the train_loader with tqdm to create a progress bar
    train_loader_with_progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch_idx, (src, trg) in enumerate(train_loader_with_progress):
        src = pad_sequence([torch.as_tensor(seq, dtype=torch.long) for seq in src], padding_value=vocab_en['<pad>']).to(device)
        trg = pad_sequence([torch.as_tensor(seq, dtype=torch.long) for seq in trg], padding_value=vocab_fr['<pad>']).to(device)

        # Forward pass
        optimizer.zero_grad()
        output = model(src, trg[:, :-1])  # Exclude the <eos> token from the input

        # Calculate the loss
        output_dim = output.shape[-1]
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:, 1:].contiguous().view(-1)  # Exclude the <sos> token from the target
        loss = criterion(output, trg)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print the average loss for this epoch
    average_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {average_loss:.4f}")


Epoch 1/4: 100%|██████████| 469/469 [1:49:51<00:00, 14.05s/it]


Epoch [1/4] - Loss: 4.9972


Epoch 2/4:   4%|▍         | 18/469 [04:30<1:52:52, 15.02s/it]


KeyboardInterrupt: ignored

## Analysis

In [ ]:
import matplotlib.pyplot as plt

# Data for analysis
datasets = ["test1.txt", "train1.txt", "dev1.txt"]
highest_bleu = [1.0, 1.0, 1.0]
max_count = [15, 276, 11]

# Create subplots
fig, ax1 = plt.subplots(figsize=(8, 6))

# Plot the highest BLEU scores
ax1.bar(datasets, highest_bleu, color='lightblue')
ax1.set_xlabel('Datasets')
ax1.set_ylabel('Highest BLEU Score', color='blue')
ax1.set_ylim(0, 1)

# Create a second y-axis for the maximum "1.0" scores
ax2 = ax1.twinx()
ax2.plot(datasets, max_count, marker='o', color='orange', linestyle='--')
ax2.set_ylabel('Max Count of "1.0" Scores', color='orange')
ax2.set_ylim(0, max(max_count) + 10)

# Add labels and a title
plt.title('Highest BLEU Scores and Max Count of "1.0" Scores')
plt.show()
